# CIFAR 10 - CNN NETWORKs:

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=False, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=32 * 8 * 8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to train the model
def train_model(optimizer_type, lr, epochs=20):
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_type == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    print(f'Training with {optimizer_type} completed.')

    return model

# Function to evaluate the model
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0

    # For calculating accuracy for individual classes
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Calculate accuracy for individual classes
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()  # Add correct predictions for this class
                class_total[label] += 1  # Increment total samples for this class

    # Overall accuracy of the model
    print(f'Overall Accuracy: {100 * correct / total:.2f}%')

    # Accuracy for each class
    for i in range(10):
        if class_total[i] > 0:  # Prevent division by zero
            print(f'Accuracy for {classes[i]}: {100 * class_correct[i] / class_total[i]:.2f}%')


# Train with SGD
print("Training with SGD:")
model_sgd = train_model(optimizer_type='SGD', lr=0.001, epochs=20)
evaluate_model(model_sgd)
print("\n")
# Train with Adam
print("\nTraining with Adam:")
model_adam = train_model(optimizer_type='Adam', lr=0.001, epochs=20)
evaluate_model(model_adam)
print("\n")
# Train with RMSprop
print("\nTraining with RMSprop:")
model_rmsprop = train_model(optimizer_type='RMSprop', lr=0.001, epochs=20)
evaluate_model(model_rmsprop)
print("\n")
# Train with AdamW
print("\nTraining with AdamW:")
model_rmsprop = train_model(optimizer_type='AdamW', lr=0.001, epochs=20)
evaluate_model(model_rmsprop)



Files already downloaded and verified
Files already downloaded and verified
Training with SGD:
Epoch [1/20], Loss: 2.2845
Epoch [2/20], Loss: 2.1889
Epoch [3/20], Loss: 2.0426
Epoch [4/20], Loss: 1.9481
Epoch [5/20], Loss: 1.8655
Epoch [6/20], Loss: 1.7841
Epoch [7/20], Loss: 1.7113
Epoch [8/20], Loss: 1.6447
Epoch [9/20], Loss: 1.5814
Epoch [10/20], Loss: 1.5334
Epoch [11/20], Loss: 1.4940
Epoch [12/20], Loss: 1.4602
Epoch [13/20], Loss: 1.4276
Epoch [14/20], Loss: 1.4015
Epoch [15/20], Loss: 1.3779
Epoch [16/20], Loss: 1.3558
Epoch [17/20], Loss: 1.3381
Epoch [18/20], Loss: 1.3151
Epoch [19/20], Loss: 1.2962
Epoch [20/20], Loss: 1.2778
Training with SGD completed.
Overall Accuracy: 53.64%
Accuracy for plane: 60.50%
Accuracy for car: 68.50%
Accuracy for bird: 38.80%
Accuracy for cat: 32.10%
Accuracy for deer: 53.80%
Accuracy for dog: 42.40%
Accuracy for frog: 59.00%
Accuracy for horse: 58.90%
Accuracy for ship: 66.20%
Accuracy for truck: 56.20%



Training with Adam:
Epoch [1/20], Los

In [13]:
class SimpleCNNWithDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=32 * 8 * 8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

        # Dropout layers
        self.dropout = nn.Dropout(p=0.5)  # 50% of neurons will be dropped

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = self.dropout(F.relu(self.fc1(x)))  # Apply dropout before the final layer
        x = self.fc2(x)
        return x

def train_model_with_dropout(optimizer_type, lr, epochs=20):
    model = SimpleCNNWithDropout().to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_type == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    print(f'Training with {optimizer_type} and Dropout completed.')
    return model

# Train with SGD
print("Training with SGD and Dropout:")
model_sgd_dropout = train_model_with_dropout(optimizer_type='SGD', lr=0.001, epochs=20)
evaluate_model(model_sgd_dropout)
print("\n")
# Train with Adam
print("\nTraining with Adam and Dropout:")
model_adam_dropout = train_model_with_dropout(optimizer_type='Adam', lr=0.001, epochs=20)
evaluate_model(model_adam_dropout)
print("\n")
# Train with RMSprop
print("\nTraining with RMSprop and Dropout:")
model_rmsprop_dropout = train_model_with_dropout(optimizer_type='RMSprop', lr=0.001, epochs=20)
evaluate_model(model_rmsprop_dropout)
print("\n")
# Train with AdamW
print("\nTraining with AdamW and Dropout:")
model_rmsprop_dropout = train_model_with_dropout(optimizer_type='AdamW', lr=0.001, epochs=20)
evaluate_model(model_rmsprop_dropout)


Training with SGD and Dropout:
Epoch [1/20], Loss: 2.2950
Epoch [2/20], Loss: 2.2615
Epoch [3/20], Loss: 2.1743
Epoch [4/20], Loss: 2.0818
Epoch [5/20], Loss: 1.9899
Epoch [6/20], Loss: 1.9024
Epoch [7/20], Loss: 1.8402
Epoch [8/20], Loss: 1.7977
Epoch [9/20], Loss: 1.7511
Epoch [10/20], Loss: 1.7161
Epoch [11/20], Loss: 1.6796
Epoch [12/20], Loss: 1.6452
Epoch [13/20], Loss: 1.6154
Epoch [14/20], Loss: 1.5882
Epoch [15/20], Loss: 1.5590
Epoch [16/20], Loss: 1.5299
Epoch [17/20], Loss: 1.5081
Epoch [18/20], Loss: 1.4879
Epoch [19/20], Loss: 1.4693
Epoch [20/20], Loss: 1.4471
Training with SGD and Dropout completed.
Overall Accuracy: 51.66%
Accuracy for plane: 57.60%
Accuracy for car: 64.60%
Accuracy for bird: 36.50%
Accuracy for cat: 30.00%
Accuracy for deer: 42.10%
Accuracy for dog: 39.70%
Accuracy for frog: 63.00%
Accuracy for horse: 57.20%
Accuracy for ship: 69.10%
Accuracy for truck: 56.80%



Training with Adam and Dropout:
Epoch [1/20], Loss: 1.7472
Epoch [2/20], Loss: 1.4392
Epo